<div align="center">

# **Predicciones de Modelos GLM**

</div>

Este notebook calcula las predicciones para todas las combinaciones posibles de variables explicativas en los modelos de frecuencia y severidad.

## Librerías y Carga de Modelos

In [1]:
library(dplyr)
library(tidyr)
library(knitr)
library(kableExtra)

# Cargar modelos entrenados
modelo_frecuencia <- readRDS("../models/modelo_frecuencia_quasipoisson.rds")
modelo_severidad <- readRDS("../models/modelo_severidad_lognormal.rds")

# Cargar datos procesados
datos <- read.csv("../data/processed/tabla_severidad.csv")
datos$X <- NULL


Attaching package: ‘dplyr’


The following objects are masked from ‘package:stats’:

    filter, lag


The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union



Attaching package: ‘kableExtra’


The following object is masked from ‘package:dplyr’:

    group_rows




## Establecer Categorías de Referencia

In [2]:
establecer_categorias_referencia <- function(df, vars_categoricas, var_exposicion = "exposicion_total") {
  df_copy <- df
  
  for (var in vars_categoricas) {
    if (var %in% colnames(df_copy)) {
      exposicion_por_categoria <- aggregate(df_copy[[var_exposicion]], 
                                          by = list(df_copy[[var]]), 
                                          FUN = sum, na.rm = TRUE)
      names(exposicion_por_categoria) <- c("categoria", "exposicion_total")
      
      categoria_referencia <- exposicion_por_categoria$categoria[which.max(exposicion_por_categoria$exposicion_total)]
      
      df_copy[[var]] <- factor(df_copy[[var]])
      df_copy[[var]] <- relevel(df_copy[[var]], ref = as.character(categoria_referencia))
    }
  }
  
  return(df_copy)
}

vars_categoricas <- c("Modelo", "Color", "Carroceria", "SERVICIO", "Sexo_Aseg", "Edad", "Gama")
datos <- establecer_categorias_referencia(datos, vars_categoricas)

# Verificar niveles de factores
cat("Niveles de variables:\n")
cat("Modelo:", levels(datos$Modelo), "\n")
cat("Edad:", levels(datos$Edad), "\n")
cat("Color:", levels(datos$Color), "\n")
cat("Carroceria:", levels(datos$Carroceria), "\n")
cat("SERVICIO:", levels(datos$SERVICIO), "\n")

Niveles de variables:
Modelo: 2007_2013 1993_1999 2000_2006 
Edad: 41_63 19_41 63_85 Otros 
Color: Otros AMARILLO BLANCO PLATA 
Carroceria: SEDAN CABINADO HATCHBACK Otros 
SERVICIO: Particular Oficial Publico Transporte Personal 


## ANÁLISIS 1: Modelo de Frecuencia (Quasi-Poisson)

Variables explicativas: **Modelo + Edad**

In [3]:
# Crear todas las combinaciones posibles para frecuencia
combinaciones_freq <- expand.grid(
  Modelo = levels(datos$Modelo),
  Edad = levels(datos$Edad),
  stringsAsFactors = FALSE
)

# Añadir exposición estándar de 1 año
combinaciones_freq$exposicion_total <- 1

cat("Combinaciones para modelo de frecuencia:", nrow(combinaciones_freq), "\n")
print(combinaciones_freq)

Combinaciones para modelo de frecuencia: 12 
      Modelo  Edad exposicion_total
1  2007_2013 41_63                1
2  1993_1999 41_63                1
3  2000_2006 41_63                1
4  2007_2013 19_41                1
5  1993_1999 19_41                1
6  2000_2006 19_41                1
7  2007_2013 63_85                1
8  1993_1999 63_85                1
9  2000_2006 63_85                1
10 2007_2013 Otros                1
11 1993_1999 Otros                1
12 2000_2006 Otros                1


In [ ]:
# Convertir a factores con los mismos niveles que los datos originales
combinaciones_freq$Modelo <- factor(combinaciones_freq$Modelo, levels = levels(datos$Modelo))
combinaciones_freq$Edad <- factor(combinaciones_freq$Edad, levels = levels(datos$Edad))

# Calcular predicciones de frecuencia
predicciones_freq <- predict(modelo_frecuencia, newdata = combinaciones_freq, type = "response")

# TABLA COMPLETA DE FRECUENCIAS - TODAS LAS COMBINACIONES
tabla_frecuencia_completa <- data.frame(
  Modelo = combinaciones_freq$Modelo,
  Edad = combinaciones_freq$Edad,
  Frecuencia_Esperada = round(predicciones_freq, 6)
) %>%
  arrange(desc(Frecuencia_Esperada))

cat("\n═══════════════════════════════════════════════════════════════════\n")
cat("      TABLA COMPLETA - MODELO DE FRECUENCIA (12 COMBINACIONES)     \n")
cat("═══════════════════════════════════════════════════════════════════\n")
cat("Variables: Modelo + Edad | Distribución: Quasi-Poisson\n")
cat("Ordenada de mayor a menor frecuencia esperada\n\n")
print(tabla_frecuencia_completa, row.names = FALSE)

# Exportar tabla para LaTeX
write.csv(tabla_frecuencia_completa, "../data/output/tabla_frecuencia_completa.csv", row.names = FALSE)

cat("\n✅ Tabla de frecuencia exportada: tabla_frecuencia_completa.csv\n")
cat("📊 Total de combinaciones:", nrow(tabla_frecuencia_completa), "\n")

In [ ]:
# Crear tabla pivot para frecuencia (mejor formato para LaTeX)
tabla_pivot_frecuencia <- tabla_frecuencia_completa %>%
  select(Modelo, Edad, Frecuencia_Esperada) %>%
  pivot_wider(names_from = Edad, values_from = Frecuencia_Esperada) %>%
  mutate_if(is.numeric, ~ round(.x, 6))

cat("\n═══════════════════════════════════════════════════════════════════\n")
cat("                  TABLA PIVOT - FRECUENCIAS                        \n")
cat("═══════════════════════════════════════════════════════════════════\n")
print(tabla_pivot_frecuencia, row.names = FALSE)

# Exportar tabla pivot para LaTeX
write.csv(tabla_pivot_frecuencia, "../data/output/tabla_pivot_frecuencia.csv", row.names = FALSE)

cat("\n✅ Tabla pivot de frecuencia exportada: tabla_pivot_frecuencia.csv\n")

## ANÁLISIS 2: Modelo de Severidad (Log-Normal)

Variables explicativas: **Modelo + Color + Carrocería + Servicio + Edad**

In [5]:
# Crear todas las combinaciones posibles para severidad
combinaciones_sev <- expand.grid(
  Modelo = levels(datos$Modelo),
  Color = levels(datos$Color),
  Carroceria = levels(datos$Carroceria),
  SERVICIO = levels(datos$SERVICIO),
  Edad = levels(datos$Edad),
  stringsAsFactors = FALSE
)

cat("Combinaciones para modelo de severidad:", nrow(combinaciones_sev), "\n")
cat("Primeras 10 combinaciones:\n")
print(head(combinaciones_sev, 10))

Combinaciones para modelo de severidad: 768 
Primeras 10 combinaciones:
      Modelo    Color Carroceria   SERVICIO  Edad
1  2007_2013    Otros      SEDAN Particular 41_63
2  1993_1999    Otros      SEDAN Particular 41_63
3  2000_2006    Otros      SEDAN Particular 41_63
4  2007_2013 AMARILLO      SEDAN Particular 41_63
5  1993_1999 AMARILLO      SEDAN Particular 41_63
6  2000_2006 AMARILLO      SEDAN Particular 41_63
7  2007_2013   BLANCO      SEDAN Particular 41_63
8  1993_1999   BLANCO      SEDAN Particular 41_63
9  2000_2006   BLANCO      SEDAN Particular 41_63
10 2007_2013    PLATA      SEDAN Particular 41_63


In [ ]:
# Convertir a factores con los mismos niveles que los datos originales
combinaciones_sev$Modelo <- factor(combinaciones_sev$Modelo, levels = levels(datos$Modelo))
combinaciones_sev$Color <- factor(combinaciones_sev$Color, levels = levels(datos$Color))
combinaciones_sev$Carroceria <- factor(combinaciones_sev$Carroceria, levels = levels(datos$Carroceria))
combinaciones_sev$SERVICIO <- factor(combinaciones_sev$SERVICIO, levels = levels(datos$SERVICIO))
combinaciones_sev$Edad <- factor(combinaciones_sev$Edad, levels = levels(datos$Edad))

# Calcular predicciones de severidad (en escala logarítmica)
predicciones_sev_log <- predict(modelo_severidad, newdata = combinaciones_sev, type = "response")

# Convertir de logaritmo a escala original
predicciones_sev <- exp(predicciones_sev_log)

# TABLA COMPLETA DE SEVERIDAD - TODAS LAS COMBINACIONES
tabla_severidad_completa <- data.frame(
  Modelo = combinaciones_sev$Modelo,
  Color = combinaciones_sev$Color,
  Carroceria = combinaciones_sev$Carroceria,
  SERVICIO = combinaciones_sev$SERVICIO,
  Edad = combinaciones_sev$Edad,
  Severidad_Esperada = round(predicciones_sev, 0)
) %>%
  arrange(desc(Severidad_Esperada))

cat("\n═══════════════════════════════════════════════════════════════════\n")
cat("      TABLA COMPLETA - MODELO DE SEVERIDAD (768 COMBINACIONES)     \n")
cat("═══════════════════════════════════════════════════════════════════\n")
cat("Variables: Modelo + Color + Carrocería + Servicio + Edad\n")
cat("Distribución: Log-Normal | Ordenada de mayor a menor severidad\n\n")

# Mostrar las primeras 30 y últimas 10 combinaciones
cat("PRIMERAS 30 COMBINACIONES (Mayor severidad):\n")
print(head(tabla_severidad_completa, 30), row.names = FALSE)

cat("\n...\n")
cat("ÚLTIMAS 10 COMBINACIONES (Menor severidad):\n")  
print(tail(tabla_severidad_completa, 10), row.names = FALSE)

# Exportar tabla completa para LaTeX
write.csv(tabla_severidad_completa, "../data/output/tabla_severidad_completa.csv", row.names = FALSE)

cat("\n✅ Tabla de severidad exportada: tabla_severidad_completa.csv\n")
cat("📊 Total de combinaciones:", nrow(tabla_severidad_completa), "\n")
cat("💰 Severidad mínima:", format(min(tabla_severidad_completa$Severidad_Esperada), big.mark = ","), "COP\n")
cat("💰 Severidad máxima:", format(max(tabla_severidad_completa$Severidad_Esperada), big.mark = ","), "COP\n")

In [ ]:
# TABLA COMPLETA DE SEVERIDAD - TODAS LAS 768 COMBINACIONES
# Crear tabla formateada para LaTeX con todas las combinaciones
tabla_severidad_768_completa <- tabla_severidad_completa %>%
  mutate(
    Ranking = row_number(),
    Severidad_LaTeX = paste0("\\$", format(Severidad_Esperada, big.mark = ","))
  ) %>%
  select(Ranking, Modelo, Color, Carroceria, SERVICIO, Edad, Severidad_Esperada, Severidad_LaTeX)

cat("\n═══════════════════════════════════════════════════════════════════\n")
cat("        TABLA COMPLETA DE SEVERIDAD - 768 COMBINACIONES            \n")
cat("═══════════════════════════════════════════════════════════════════\n")
cat("Para insertar en documentación LaTeX\n\n")

# Mostrar estructura de la tabla
cat("Estructura de la tabla (primeras 20 filas):\n")
print(head(tabla_severidad_768_completa, 20), row.names = FALSE)

cat("\n...")
cat("\nÚltimas 5 filas:\n")
print(tail(tabla_severidad_768_completa, 5), row.names = FALSE)

# Exportar tabla completa para LaTeX
write.csv(tabla_severidad_768_completa, "../data/output/tabla_severidad_768_completa.csv", row.names = FALSE)

cat("\n✅ Tabla completa de 768 combinaciones exportada: tabla_severidad_768_completa.csv\n")
cat("📊 Combinaciones ordenadas de mayor a menor severidad esperada\n")

## ANÁLISIS 3: Cálculo de Primas Puras

Para el Top 15 de combinaciones de severidad, calcular la prima pura correspondiente

In [ ]:
# Obtener las top 15 combinaciones de severidad
top15_severidad <- tabla_severidad_completa[1:15, ]

# Para cada combinación del top 15, obtener su frecuencia correspondiente
# Solo necesitamos Modelo y Edad para el modelo de frecuencia
primas_puras <- data.frame()

for(i in 1:nrow(top15_severidad)) {
  # Obtener modelo y edad de la combinación actual
  modelo_actual <- top15_severidad$Modelo[i]
  edad_actual <- top15_severidad$Edad[i]
  
  # Buscar la frecuencia correspondiente en la tabla de frecuencias
  frecuencia_correspondiente <- tabla_frecuencia_completa[
    tabla_frecuencia_completa$Modelo == modelo_actual & 
    tabla_frecuencia_completa$Edad == edad_actual, 
    "Frecuencia_Esperada"
  ]
  
  # Calcular prima pura = frecuencia × severidad
  prima_pura <- frecuencia_correspondiente * top15_severidad$Severidad_Esperada[i]
  
  # Agregar a la tabla de resultados
  primas_puras <- rbind(primas_puras, data.frame(
    Rank = i,
    Modelo = as.character(top15_severidad$Modelo[i]),
    Color = as.character(top15_severidad$Color[i]),
    Carroceria = as.character(top15_severidad$Carroceria[i]),
    SERVICIO = as.character(top15_severidad$SERVICIO[i]),
    Edad = as.character(top15_severidad$Edad[i]),
    Severidad_Esperada = top15_severidad$Severidad_Esperada[i],
    Frecuencia_Esperada = frecuencia_correspondiente,
    Prima_Pura = round(prima_pura, 0)
  ))
}

cat("═══════════════════════════════════════════════════════════════════\n")
cat("              TABLA DE PRIMAS PURAS - TOP 15                       \n")
cat("═══════════════════════════════════════════════════════════════════\n")
cat("Prima Pura = Frecuencia Esperada × Severidad Esperada\n\n")

print(primas_puras, row.names = FALSE)

# Exportar tabla de primas puras
write.csv(primas_puras, "../data/output/tabla_primas_puras_top15.csv", row.names = FALSE)

cat("\n✅ Tabla de primas puras exportada: tabla_primas_puras_top15.csv\n")
cat("💰 Prima mínima (Top 15):", format(min(primas_puras$Prima_Pura), big.mark = ","), "COP\n")
cat("💰 Prima máxima (Top 15):", format(max(primas_puras$Prima_Pura), big.mark = ","), "COP\n")
cat("📊 Ratio máximo/mínimo:", round(max(primas_puras$Prima_Pura) / min(primas_puras$Prima_Pura), 2), "veces\n")